In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import between
from home_messages_db import HomeMessagesDB
from functions import create_timestamp, df_timestamp_index

In [4]:
import sqlalchemy as sa
import pandas as pd
engine = sa.create_engine("sqlite:///myhome.db")
sql = sa.text("SELECT * FROM p1g")
df = pd.read_sql(sql, con=engine)
df.head()

OperationalError: (sqlite3.OperationalError) no such table: p1g
[SQL: SELECT * FROM p1g]
(Background on this error at: https://sqlalche.me/e/14/e3q8)